# Introduction
In this exercise, a neural network will be created. This will be done for the famous MNIST dataset, which contains thousands of images with the a letter from 0-9 and their corresponding label.

The neural network will be created based on object oriented programming. Meaning, that a class will be created for the architecture of the model.

In [1]:
# Install idx2numpy and torch if you haven't already
#%pip install idx2numpy
#%pip install torch

In [2]:
# Import the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import idx2numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Importing the data and making it ready
In this chapter, the data will be imported and prepared for training and testing the neural network. The data is retrieved from the following website: http://yann.lecun.com/exdb/mnist/

In [3]:
# Read the datasets as ubyte files and transform them into 2/3-dimensional numpy arrays
train_images = idx2numpy.convert_from_file("train-images.idx3-ubyte")
train_labels = idx2numpy.convert_from_file("train-labels.idx1-ubyte")
test_images = idx2numpy.convert_from_file("t10k-images.idx3-ubyte")
test_labels = idx2numpy.convert_from_file("t10k-labels.idx1-ubyte")

train_images.shape

(60000, 28, 28)

## Making the data ready
In this chapter, the data will be prepared for the neural network. First, the data will be made ready for the `Dataloader` function. This will be done by creating a class `Data`, which inherits the Data `torch.utils.data.Dataset` properties. Then, the data will be scaled so that it ranges from 0-1, instead of 0-255. And finally the datasets will be transformed into a DataLoader, which will be used to train and test the neural network.

In [4]:
# Define a new class which makes the data ready for the DataLoader.
# This is done by inheriting the torch.utils.data.Dataset properties
class Data(Dataset):
    # init function which takes X_train and y_train data
    def __init__(self, X_train, y_train):
        # need to convert float64 to float32 else 
        # will get the following error
        # RuntimeError: expected scalar type Double but found Float
        self.X = torch.from_numpy(X_train.astype(np.float32))
        # need to convert float64 to Long else 
        # will get the following error
        # RuntimeError: expected scalar type Long but found Float
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]
  
    # Function to retrieve the value of X and y based on the index
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    # Function to retrieve the length of the dataset
    def __len__(self):
        return self.len

In [5]:
# Create a function which scales the input data into 0 and 1
def min_max_scaled(dataset):
    # Manually scale the data between 0 and 1
    data = ((dataset - dataset.min()) / dataset.max() - dataset.min())
    
    return data

In [6]:
# Scale the data so that is has a range between 0 and 1
# This is done manually to avoid for looping
train_images_scaled = min_max_scaled(train_images)
test_images_scaled = min_max_scaled(test_images)

# Define the train and test set using the Data class we created earlier
trainset = Data(train_images_scaled, train_labels)
testset = Data(test_images_scaled, test_labels)

# Create DataLoaders using DataLoader class: trainLoader, testLoader
trainloader = DataLoader(trainset, batch_size = 10,
                        shuffle = True, num_workers=0)
testloader = DataLoader(testset, batch_size = 10,
                        shuffle = False, num_workers=0)

C:\Users\ardejong\AppData\Local\Temp/ipykernel_21676/2323167276.py:13: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  self.y = torch.from_numpy(y_train).type(torch.LongTensor)


# Neural network
In this chapter, the neural network will be created, trained and tested. For the neural network, the architecture of the model looks like:
- An input layer with 28 * 28 * 1 nodes: 784
- A 1st hidden layer with 14 * 14 nodes: 196
- A 2nd hidden layer with 7 * 7 nodes: 28
- An output layer with ten nodes

This architecture is chosen because the shape of a image is 28 * 28 * 1. After the input layer, we devide the shape in half, so the 1st hidden layer would be 14 * 14. This is done for the 2nd layer as well. This is done, so that splitting is halved each layer.

There are 10 output nodes, since there are 10 possible outputs for the labels: a number between 0 and 9.

The metric to measure our performance is `accuracy`. This metric is chosen because it fits our machine learning type the best: multi-classification. Metrics like loss is more suitable for regression problems.

In [7]:
# Define the neural network using OOP: Net
# It inherits the torch.nn.Module properties
class Net(nn.Module):
    def __init__(self):    
    # Define all the parameters of the net
        super(Net, self).__init__()
        
        # Define the layers for the neural network
        self.fc1 = nn.Linear(28 * 28 * 1, 14 * 14)
        self.fc2 = nn.Linear(14 * 14, 28)
        self.fc3 = nn.Linear(28, 10)

    def forward(self, x):   
    # Do the forward pass
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [8]:
# Instantiate model
model = Net()

# Define loss and optimzer functions
# Choose learning rate of 3e-4
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=3e-4)
print(model.parameters)

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=196, bias=True)
  (fc2): Linear(in_features=196, out_features=28, bias=True)
  (fc3): Linear(in_features=28, out_features=10, bias=True)
)>


## Training the model
In this chapter, the model will be trained. For the training, we chose 15 epochs, because during development, we found out that the accuracy will go down if we chose more.

In [9]:
# Train the data using a for loop
# Define number of epochs
epochs = 15

accuracies = []
for epoch in range(epochs):
    # Initialize total and correct to 0
    # These will be used later
    total = 0
    correct = 0
    
    for data in trainloader:
        inputs, labels = data
        
        # Put each image into a vector
        inputs = inputs.view(-1, 28 * 28)
        
        # set optimizer to zero grad to remove previous epoch gradients
        optim.zero_grad()
        
        # Apply forward propagation using the model
        # Get loss function for data
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, outputs = torch.max(outputs.data, 1)
        
        # Add the total to the total variable
        total += labels.size(0)
        
        # Add the correct to the correct variable
        correct += (outputs == labels).sum()
        
        # backward propagation
        loss.backward()
        
        # optimize
        optim.step()
    
    # Print the accuracy of the epoch
    accuracy = (correct / total) * 100
    print(f"Accuracy epoch {epoch + 1}: {accuracy:.3f}%")

Accuracy epoch 1: 87.787%
Accuracy epoch 2: 94.000%
Accuracy epoch 3: 95.458%
Accuracy epoch 4: 96.460%
Accuracy epoch 5: 97.035%
Accuracy epoch 6: 97.465%
Accuracy epoch 7: 97.803%
Accuracy epoch 8: 98.063%
Accuracy epoch 9: 98.297%
Accuracy epoch 10: 98.450%
Accuracy epoch 11: 98.598%
Accuracy epoch 12: 98.710%
Accuracy epoch 13: 98.810%
Accuracy epoch 14: 98.950%
Accuracy epoch 15: 99.022%


In [13]:
# Set the model in eval mode
model.eval()

# Initialize total and correct to 0
# These will be used later
total = 0
correct = 0

# Predict for each row in the test set and see if there correct
for data in testloader:
    inputs, labels = data
    
    # Put each image into a vector
    inputs = inputs.view(-1, 28 * 28 * 1)
    
    # Do the forward pass and get the predictions
    outputs = model(inputs)
    _, outputs = torch.max(outputs.data, 1)
    
    # Add the total to the total variable
    total += labels.size(0)
    
    # Add the correct to the correct variable
    correct += (outputs == labels).sum()
    
    
accuracy = (correct / total) * 100   
print(f'The testing set accuracy of the neural network is: {accuracy:.3f}%')

The testing set accuracy of the neural network is: 98.020%


# Conclusion

Based on the accuracy of both the train and test set, we can conclude that the model performs really great. For the train set, right on the first epoch, we have an accuracy of 87.787%, and 99.022% after the final epochs. For the test set, we have an accuracy of 98.020%. Based on the fact that the accuracies from the train and test set only differ around 1 percent, we can conclude that the model is well trained (neither over- or undertrained).